In [1]:
import pandas as pd
import numpy as np
from torcheval.metrics.functional import binary_precision_recall_curve
import torch
from proteinfertorch.utils import read_pickle

In [37]:
from collections import defaultdict
model_ids = defaultdict(list)
file_name2var_name = {
    "noxpd2_cnn_swissprot_go_clustered_swiss-cnn_for_swissprot_go_clustered":"GO_CLUSTERED_ENSEMBLE_ELEMENT_EXPERIMENT_IDS",
    "noxpd2_cnn_swissprot_go_random_swiss-cnn_for_swissprot_go_random":"GO_RANDOM_ENSEMBLE_ELEMENT_EXPERIMENT_IDS",
    "noxpd2_cnn_swissprot_ec_clustered_swiss-cnn_for_swissprot_ec_clustered":"EC_CLUSTERED_ENSEMBLE_ELEMENT_EXPERIMENT_IDS",
    "noxpnd_cnn_swissprot_ec_random_swiss-cnn_for_swissprot_ec_random":"EC_RANDOM_ENSEMBLE_ELEMENT_EXPERIMENT_IDS",

}

for i in pd.read_csv('../zipped_models.txt', header=None).values.flatten():
    file_name = i.split('/')[-1].replace('.tar.gz', '')
    #Only consider file_names with go or ec in them
    if '-'.join(file_name.split('-')[:-1]) in file_name2var_name:
        # Split string noxpd2_cnn_swissprot_ec_clustered_swiss-cnn_for_swissprot_ec_clustered-13704042.tar.gz' by - followed by number and extension
        split = file_name.split('-')
        w_id = int(split[-1])
        name = '-'.join(split[:-1])
        model_ids[file_name2var_name[name]].append(str(w_id))


In [ ]:
labels = pd.read_hdf("../outputs/test_labels_ProteInfer.h5", key="labels_df")
probabilities = pd.read_hdf("../outputs/test_probabilities_ProteInfer.h5", key="probabilities_df")

logits_binary = torch.tensor(probabilities.values.flatten(),device='cuda')
labels_binary = torch.tensor(labels.values.flatten(),device='cuda')
precision, recall, thresholds = binary_precision_recall_curve(logits_binary, labels_binary)
f1 = 2 * precision * recall / (precision + recall)

best_th,best_f1 = thresholds[torch.argmax(f1)].item(),torch.max(f1).item()
print(best_th,best_f1)